Brandon Connors  
11/5/23  
Practical Dashboards

### Pipeline Code for Professor's Review

```
[
    {
        '$project': {
            'id': '$id', 
            'title': '$title', 
            'runtime': {
                '$convert': {
                    'input': '$runtime', 
                    'to': 'int', 
                    'onError': None
                }
            }, 
            'user_rating': {
                '$convert': {
                    'input': '$user_rating', 
                    'to': 'double', 
                    'onError': None
                }
            }, 
            'votes': {
                '$convert': {
                    'input': '$votes', 
                    'to': 'long', 
                    'onError': None
                }
            }, 
            'mpaa_rating': {
                '$split': [
                    '$mpaa_rating', '::'
                ]
            }, 
            'release_date': {
                '$convert': {
                    'input': '$release_date', 
                    'to': 'date', 
                    'onError': None
                }
            }, 
            'budget': {
                '$convert': {
                    'input': '$budget', 
                    'to': 'long', 
                    'onError': None
                }
            }, 
            'opening_weekend': {
                '$convert': {
                    'input': '$opening_weekend', 
                    'to': 'long', 
                    'onError': None
                }
            }, 
            'gross_sales': {
                '$convert': {
                    'input': '$gross_sales', 
                    'to': 'long', 
                    'onError': None
                }
            }, 
            'genres': {
                '$split': [
                    {
                        '$toString': '$genres'
                    }, ', '
                ]
            }, 
            'cast': {
                '$split': [
                    {
                        '$toString': '$cast'
                    }, ', '
                ]
            }, 
            'director': {
                '$split': [
                    {
                        '$toString': '$director'
                    }, ', '
                ]
            }, 
            'producer': {
                '$split': [
                    {
                        '$toString': '$producer'
                    }, ', '
                ]
            }, 
            'company': {
                '$split': [
                    {
                        '$toString': '$company'
                    }, ', '
                ]
            }
        }
    }, {
        '$addFields': {
            'slimmed_rating': {
                '$arrayElemAt': [
                    '$mpaa_rating', 0
                ]
            }, 
            'release_year': {
                '$year': '$release_date'
            }, 
            'release_month': {
                '$month': '$release_date'
            }
        }
    }
]
```

### Importing the Connection String

In [1]:
import json

# Loading the json file that contains the connection string
with open('D:\\School\\Fall 2023\\secrets.json') as d:
    data = json.load(d)

    # Extracting the connection string from the file
    connectionString = data['mongodbRead']

# Sanity check to make sure that I've pulled the right data
print(f"My connection string is {len(connectionString)} characters in length.")

My connection string is 73 characters in length.


### Connecting to the Database and Verifying the Connection

In [2]:
# I copied this section of code from the professor's sample code labeled "Basics of Loading Data" and then modified it to work with my connection string above

import pymongo
import certifi

# Connect to the database using known good certificates
client = pymongo.MongoClient(connectionString, tlsCAFile=certifi.where())

# Fetch the database named "DA320"
da320_database = client['DA320']

# Within the database we have "collections". Think of them as tables in SQL.
allCollections = da320_database.list_collection_names()

# Here is the list of collections within my database
print(f"Using MongoDB version {client.server_info()['version']}.")
print(f"This database has the collections {allCollections}")

Using MongoDB version 6.0.11.
This database has the collections ['MetacriticMovies', 'IMDB_Superhero_2', 'IMDB_Pipeline_View']


### Loading the Data

In [3]:
# Again, here I copied the code from the professor and modified it to extract just a single year's worth of data

import pandas as pd

# Here I've got the query developed in MongoDB Compass and then exported to Python code using the export function
query = {'release_year': 2005}

# Execute this query and produce a cursor
cursor = da320_database["IMDB_Pipeline_View"].find(query)

# Convert this information into a Pandas dataframe
imdbFinal = pd.DataFrame(cursor)

# Checking the number of results
print(f"My resulting table has {len(imdbFinal.title)} results")

# Make sure we've read the information correctly
imdbFinal.head()

My resulting table has 817 results


,_id,id,title,runtime,user_rating,votes,mpaa_rating,release_date,budget,opening_weekend,gross_sales,genres,cast,director,producer,company,slimmed_rating,release_year,release_month
0,652cd358f089e541968ca9be,414387,Pride & Prejudice,129.0,7.8,289263,[PG],2005-07-25,28000000.0,NaN,121147947.0,"[Drama, Romance]","[Keira Knightley, Talulah Riley, Rosamund Pike...",[Joe Wright],"[Tim Bevan, Liza Chasin, Eric Fellner, Jane Fr...","[Focus Features, Universal Pictures, StudioCan...",PG,2005,7
1,652cd358f089e541968ca9cf,401792,Sin City,124.0,8.0,761770,[R],2005-03-28,40000000.0,29120273.0,158753820.0,"[Crime, Thriller]","[Jessica Alba, Devon Aoki, Alexis Bledel, Powe...","[Frank Miller, Quentin Tarantino, Robert Rodri...","[Elizabeth Avellan, Bill Scott, Bob Weinstein,...","[Dimension Films, Troublemaker Studios]",R,2005,3
2,652cd358f089e541968ca9d2,399146,A History of Violence,96.0,7.4,236845,[R],2005-05-16,32000000.0,515992.0,61477797.0,"[Action, Crime, Drama, Thriller]","[Viggo Mortensen, Maria Bello, Ed Harris, Will...",[David Cronenberg],"[Kent Alterman, Chris Bender, Cale Boyter, Jos...","[New Line Cinema, BenderSpink, Media I! Filmpr...",R,2005,5
3,652cd358f089e541968ca9d5,320661,Kingdom of Heaven,144.0,7.2,284768,[R],2005-05-02,130000000.0,NaN,211652051.0,"[Action, Adventure, Drama, History, War]","[Martin Hancock, Michael Sheen, Nathalie Cox, ...",[Ridley Scott],"[Karim Abouobayd, Mark Albela, Bruce Devan, Li...","[Twentieth Century Fox, Scott Free Productions...",R,2005,5
4,652cd358f089e541968ca9d8,358082,Robots,91.0,6.4,144984,[PG],2005-03-10,75000000.0,36045301.0,260718330.0,"[Animation, Adventure, Comedy, Family, Romance...","[Paula Abdul, Halle Berry, Lucille Bliss, Terr...","[Chris Wedge, Carlos Saldanha]","[Jerry Davis, John C. Donkin, Bob Gordon, Will...","[Twentieth Century Fox Animation, Blue Sky Stu...",PG,2005,3
